# Processamento dos Datasets: CASIAWeb
https://colab.research.google.com/drive/1H2UJmi7nX8j3o89PQQOL1ySGQMY8hieZ#scrollTo=qk4R9zGkKWOp

Para configurar as credenciais e ambiente do Kaggle veja: https://www.kaggle.com/discussions/general/74235

In [1]:
! pip install -q kaggle

In [3]:
! kaggle datasets download debarghamitraroy/casia-webface

Dataset URL: https://www.kaggle.com/datasets/debarghamitraroy/casia-webface
License(s): DbCL-1.0


In [4]:
import zipfile
import os

zip_path = 'casia-webface.zip'
extract_path = 'casia-webfac'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [9]:
!pip cache purge
!python -m pip install --upgrade pip

Files removed: 38
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [1]:
!pip install mxnet numpy==1.23.5 vit_pytorch timm einops tensorboardX fvcore pyyaml efficientnet_pytorch kagglehub pandas scipy scikit-learn

In [ ]:
import mxnet as mx
import os
import cv2

rec_path = 'casia-webface/casia-webface/train.rec'
idx_path = 'casia-webface/casia-webface/train.idx'
output_dir = 'extracted_faces'

os.makedirs(output_dir, exist_ok=True)

record = mx.recordio.MXIndexedRecordIO(idx_path, rec_path, 'r')

# Extrair todas as imagens
i = 0
while True:
    try:
        item = record.read_idx(i)
        if item is None:
            break

        header, img = mx.recordio.unpack(item)
        label = int(header.label) if not isinstance(header.label, list) else int(header.label[0])

        # Converte imagem para formato OpenCV
        img = mx.image.imdecode(img).asnumpy()
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        # Salva na pasta de acordo com o label
        label_folder = os.path.join(output_dir, str(label))
        os.makedirs(label_folder, exist_ok=True)
        cv2.imwrite(os.path.join(label_folder, f"{i}.jpg"), img)

        if i % 1000 == 0:
            print(f"Extraído {i} imagens...")

        i += 1

    except Exception as e:
        print(f"Erro no índice {i}: {e}")
        i += 1
        continue

print(f"Extração completa: {i} imagens")


A saída de streaming foi truncada nas últimas 5000 linhas.
Erro no índice 8352644: 8352644
Erro no índice 8352645: 8352645
Erro no índice 8352646: 8352646
Erro no índice 8352647: 8352647
Erro no índice 8352648: 8352648
Erro no índice 8352649: 8352649
Erro no índice 8352650: 8352650
Erro no índice 8352651: 8352651
Erro no índice 8352652: 8352652
Erro no índice 8352653: 8352653
Erro no índice 8352654: 8352654
Erro no índice 8352655: 8352655
Erro no índice 8352656: 8352656
Erro no índice 8352657: 8352657
Erro no índice 8352658: 8352658
Erro no índice 8352659: 8352659
Erro no índice 8352660: 8352660
Erro no índice 8352661: 8352661
Erro no índice 8352662: 8352662
Erro no índice 8352663: 8352663
Erro no índice 8352664: 8352664
Erro no índice 8352665: 8352665
Erro no índice 8352666: 8352666
Erro no índice 8352667: 8352667
Erro no índice 8352668: 8352668
Erro no índice 8352669: 8352669
Erro no índice 8352670: 8352670
Erro no índice 8352671: 8352671
Erro no índice 8352672: 8352672
Erro no índic

In [1]:
import os
from collections import defaultdict

dataset_path = 'extracted_faces'  # sua pasta extraída

# Conta imagens por classe
class_image_counts = defaultdict(int)

for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_dir):
        num_images = len([img for img in os.listdir(class_dir) if img.lower().endswith(('.jpg', '.png'))])
        class_image_counts[class_name] = num_images

# Visualizar o que temos
print(f"Número de classes: {len(class_image_counts)}")
print(f"Exemplo de contagem: {list(class_image_counts.items())[:5]}")

Número de classes: 10572
Exemplo de contagem: [('9841', 37), ('4391', 30), ('2279', 34), ('8298', 121), ('930', 48)]


In [4]:
# Ordena as classes pela quantidade de imagens (do maior para o menor)
sorted_classes = sorted(class_image_counts.items(), key=lambda x: x[1], reverse=True)

selected_classes = []
total_images = 0
limit = 15000

for class_name, num_images in sorted_classes:
    if total_images + num_images > limit:
        continue  # pula se passar de 7000
    selected_classes.append(class_name)
    total_images += num_images

print(f"Selecionadas {len(selected_classes)} classes com {total_images} imagens no total")

Selecionadas 25 classes com 14999 imagens no total


In [5]:
import shutil

output_dir = 'filtered_faces'
os.makedirs(output_dir, exist_ok=True)

for class_name in selected_classes:
    src_class_dir = os.path.join(dataset_path, class_name)
    dst_class_dir = os.path.join(output_dir, class_name)
    os.makedirs(dst_class_dir, exist_ok=True)

    for img_name in os.listdir(src_class_dir):
        src_img_path = os.path.join(src_class_dir, img_name)
        dst_img_path = os.path.join(dst_class_dir, img_name)
        shutil.copy(src_img_path, dst_img_path)

print("Cópia concluída.")

Cópia concluída.


In [6]:
import os
import cv2

# Caminho da pasta com as imagens filtradas
input_dir = 'filtered_faces'
# Caminho para salvar as imagens tratadas
output_dir = 'final_faces'
os.makedirs(output_dir, exist_ok=True)

# Carrega o Haar Cascade para detecção de rostos
haar_cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_cascade_path)

SIZE=(112,112)

# Loop sobre cada classe
for class_name in os.listdir(input_dir):
    class_input_path = os.path.join(input_dir, class_name)
    class_output_path = os.path.join(output_dir, class_name)

    if not os.path.isdir(class_input_path):
        continue

    os.makedirs(class_output_path, exist_ok=True)

    for img_name in os.listdir(class_input_path):
        img_path = os.path.join(class_input_path, img_name)
        img = cv2.imread(img_path)

        if img is None:
            print(f"Erro ao carregar imagem: {img_path}")
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        if len(faces) == 0:
            print(f"Nenhuma face encontrada: {img_path}")
            continue

        # Seleciona a maior face detectada (em caso de múltiplas)
        x, y, w, h = max(faces, key=lambda box: box[2] * box[3])

        # Recorta a face
        face_img = img[y:y+h, x:x+w]

        # Redimensiona para 112x112
        face_resized = cv2.resize(face_img, SIZE)

        # Salva a imagem normalizada
        output_img_path = os.path.join(class_output_path, img_name)
        cv2.imwrite(output_img_path, face_resized)

print("Processamento completo!")

Nenhuma face encontrada: filtered_faces/138/18021.jpg
Nenhuma face encontrada: filtered_faces/138/18147.jpg
Nenhuma face encontrada: filtered_faces/138/17923.jpg
Nenhuma face encontrada: filtered_faces/138/18115.jpg
Nenhuma face encontrada: filtered_faces/138/18358.jpg
Nenhuma face encontrada: filtered_faces/138/17928.jpg
Nenhuma face encontrada: filtered_faces/138/18093.jpg
Nenhuma face encontrada: filtered_faces/138/18513.jpg
Nenhuma face encontrada: filtered_faces/138/18337.jpg
Nenhuma face encontrada: filtered_faces/138/18274.jpg
Nenhuma face encontrada: filtered_faces/138/18341.jpg
Nenhuma face encontrada: filtered_faces/138/18009.jpg
Nenhuma face encontrada: filtered_faces/138/18196.jpg
Nenhuma face encontrada: filtered_faces/138/18366.jpg
Nenhuma face encontrada: filtered_faces/138/18255.jpg
Nenhuma face encontrada: filtered_faces/138/17857.jpg
Nenhuma face encontrada: filtered_faces/138/18233.jpg
Nenhuma face encontrada: filtered_faces/138/18300.jpg
Nenhuma face encontrada: fil

In [7]:
import shutil

# Pasta que você quer comprimir
folder_to_zip = 'final_faces'
# Nome do arquivo zip de saída
output_zip = 'processed_casiaweb.zip'

# Cria o ZIP
shutil.make_archive(base_name=folder_to_zip, format='zip', root_dir=folder_to_zip)

print(f"Pasta '{folder_to_zip}' comprimida como '{output_zip}' com sucesso!")

Pasta 'final_faces' comprimida como 'processed_casiaweb.zip' com sucesso!
